In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/kobo2020'
%ls


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/kobo2020
AttentionSeq2seq.pkl                neko.txt
attention_seq2seq.py                nlp100_chap9
BetterRnnlm.pkl                     nlp100_chap9.pkl
better_rnnlm.py                     nlp100_chap9.txt
cbow_params.pkl                     __pycache__/
cbow.py                             PyTorchTutorial1.ipynb
ch04/                               repo10.ipynb
ch06/                               repo10test.ipynb
ch0

In [32]:
# -*- coding: utf-8 -*-
"""kobo_bert.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1pwU35uTpZq29_4kCFPfrTaC2mvo_eZnK
"""

!pip install transformers
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5


Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
xz-utils is already the newest version (5.2.2-1.3).
curl is already the newest version (7.58.0-2ubuntu3.9).
file is already the newest version (1:5.32-2ubuntu0.4).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic is already the newest version (2.7.0-20070801+main-1).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
The following package was automatically installed and is no longer required:
  libnvidia-comm

In [33]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import transformers

#東北大学の
pretrained_model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# 事前学習済みモデルのトークナイザを使用
tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model_name)

# 形態素解析 (内部でMeCabを使用)
text1 = "今日はいい天気だね"
text2 = "明日は雨がふるかもしれませんね"

print("text1", tokenizer.tokenize(text1))
print("text2", tokenizer.tokenize(text2))

"""
text1 ['今日', 'は', 'いい', '天気', 'だ', 'ね']
text2 ['明日', 'は', '雨', 'が', 'ふる', 'かも', 'しれ', 'ませ', 'ん', 'ね']
"""

# BERTに入力する形式に変換
for_bert_inputs = tokenizer([text1, text2], padding=True, return_tensors="pt")#[text1,test2]でバッチ化、ptは
print("for_bert_inputs", for_bert_inputs)
"""
2はCSLトークン、0はパディング、3はセパレートSEP
for_bert_inputs {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3,     0,     0,
             0,     0],
        [    2, 11475,     9,  3741,    14,  8491,  4830,  6758,  6769,  1058,
          1852,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
"""
# input_ids: 単語をIDに変換した系列．padding済み
# token_type_ids： 2文からなるペアを入力した場合に，1文目と2文目を区別するための系列
# attention_mask： input_idsのpadding部分とそうでない部分を区別するための系列


# 参考．文のペアを入れる場合の例
text3 = "そうかな"
text4 = "違うと思います"
tmp = tokenizer([[text1, text2],[text3, text4]], padding=True, return_tensors="pt")
print("tmp", tmp)
"""
tmp {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3, 11475,     9,
          3741,    14,  8491,  4830,  6758,  6769,  1058,  1852,     3],
        [    2,  1778,    29,    18,     3,  5720,    13,  2502,  2610,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],#0が一文目、1が二文目という意味
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
"""


# BERTの事前学習済みモデルをロード．BertForSequenceClassificationは1文が与えられて分類を行うクラス．num_labelsでラベル数を指定
# 他にもいろいろ用意されている．使用例も書かれている→  https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
model = transformers.BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=5)

out = model(input_ids=for_bert_inputs["input_ids"], token_type_ids=for_bert_inputs["token_type_ids"], attention_mask=for_bert_inputs["attention_mask"])
print("out", out)

model = transformers.BertModel.from_pretrained(pretrained_model_name)
out = model(input_ids=for_bert_inputs["input_ids"], token_type_ids=for_bert_inputs["token_type_ids"], attention_mask=for_bert_inputs["attention_mask"])
# pooled_output = outputs[1]

# pooled_output = self.dropout(pooled_output)
# logits = self.classifier(pooled_output)
print("out", out[1].size())
"""
out (tensor([[-0.4694, -0.2888,  0.1584,  0.1443,  0.2474],
        [-0.4795, -0.2614,  0.0915,  0.1614,  0.1209]],
       grad_fn=<AddmmBackward>),)
"""

# あとは損失を計算していつもどおりbackpropすればOK


text1 ['今日', 'は', 'いい', '天気', 'だ', 'ね']
text2 ['明日', 'は', '雨', 'が', 'ふる', 'かも', 'しれ', 'ませ', 'ん', 'ね']
for_bert_inputs {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3,     0,     0,
             0,     0],
        [    2, 11475,     9,  3741,    14,  8491,  4830,  6758,  6769,  1058,
          1852,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tmp {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3, 11475,     9,
          3741,    14,  8491,  4830,  6758,  6769,  1058,  1852,     3],
        [    2,  1778,    29,    18,     3,  5720,    13,  2502,  2610,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

out (tensor([[-0.3551, -0.1342, -0.0980, -0.4630, -0.1572],
        [-0.3949, -0.1713, -0.0507, -0.3602, -0.2726]],
       grad_fn=<AddmmBackward>),)
out torch.Size([2, 768])


'\nout (tensor([[-0.4694, -0.2888,  0.1584,  0.1443,  0.2474],\n        [-0.4795, -0.2614,  0.0915,  0.1614,  0.1209]],\n       grad_fn=<AddmmBackward>),)\n'

In [ ]:
#データを得る
!wget http://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar xfz ldcc-20140209.tar.gz


In [25]:

category2id = {"kaden-channel":0, "peachy":1, "sports-watch":2, "dokujo-tsushin":3, "livedoor-homme":4, "it-life-hack":5, "movie-enter":6}

# Livedoorニュースコーパスからテキスト分類用のデータを作成．
import glob
import random

# 1行は [文][TAB][ラベル]からなる
write_lines = []
for d in category2id.keys():
  for file in glob.glob("text/" + d + "/*.txt"):
    with open(file) as f:
      lines = f.readlines()
      # 最初の2行はURLと日付なので捨てる
      for line in lines[3:]:
        line = line.strip()
        if len(line) > 20 and len(line) < 256 and "http" not in line:
          write_lines.append(line + "\t" + str(category2id[d]) + "\n")

random.shuffle(write_lines)
# Train, Dev, Testの3つに分ける.
dev = write_lines[0:2000]
test =write_lines[2000:4000]
train = write_lines[4000:]
# ファイルに保存
w = open("train.tsv", "w")
w.writelines(train)
w.close()
w = open("dev.tsv", "w")
w.writelines(dev)
w.close()
w = open("test.tsv", "w")
w.writelines(test)
w.close()
print("data size ", len(train), len(dev), len(test))

#!head dev.tsv
#データセットだけ作成する、Inputids、tokenids、idsの三つを返せば良い

data size  67653 2000 2000


In [35]:

import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence
#word2id_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/nlp100_chap9'#todo 80.pyで保存したファイルパスを指定
import numpy as np

    
def get_data(fname):
    inputids_list = [] # inputidsを格納するリスト
    tokenids_list = []#
    attentionmask_list = []
    ids_list = [] #label
    i = 0
    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.split('\t')#todo lineからタイトルを取得
            title = line[0]
            tmp = tokenizer(title, padding=True, return_tensors="pt")
            inputids = tmp["input_ids"]
            i += 1
            tokenids = tmp["token_type_ids"]+i
            attentionmask = tmp["attention_mask"]
            ids = float(line[1])
            
            inputids_list.append(inputids)
            tokenids_list.append(tokenids)
            attentionmask_list.append(attentionmask)
            ids_list.append(ids)

    #inputids_list = torch.from_numpy(np.array(inputids_list))
    #tokenids_list = torch.from_numpy(np.array(token_ids_list))
    ids_list = torch.from_numpy(np.array(ids_list))

    return  inputids_list,tokenids_list,attentionmask_list,ids_list

#todo データセットだけ作成する、Inputids、tokenids、idsの三つを返せば良い
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,inputids_list, tokenids_list, attentionmask_list,ids_list):
      self.length = len(ids_list)
      self.inputids_list = inputids_list
      self.tokenids_list = tokenids_list
      self.attentionmask_list = attentionmask_list
      self.ids_list = ids_list


    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.inputids_list[idx],self.tokenids_list[idx],attentionmask_list[idx],self.ids_list[idx]


In [36]:
from torch.utils.data import Dataset, DataLoader

# データの準備
train_path =  '/content/gdrive/My Drive/Colab Notebooks/kobo2020/dev.tsv'#todo 学習用ファイルパス
inputids_list, token_ids_list ,attentionmask_list,ids_list = get_data(train_path)
#print(inputids_list[0])

#train_x はテキストのリスト、yはテンソルにしてもOK
#get_dataはテキストのリストとyはテンソルを返す

dataset = MyDataset(inputids_list, token_ids_list,attentionmask_list,ids_list)
print(dataset[0])
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


(tensor([[    2,  6571,   486,    12,     6,    31,  8141,  7111,    14,     6,
         12272,    45,    11,  3083,    16,    21,    10,     8,     3]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), tensor(3., dtype=torch.float64))


In [38]:

emb_dim = 300
hidden_dim = 50
target_size = 4
batch_size = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データの準備
train_path =  '/content/gdrive/My Drive/Colab Notebooks/kobo2020/dev.tsv'#
inputids_list, token_ids_list ,attentionmask_list,ids_list = get_data(train_path)


dataset = MyDataset(inputids_list, token_ids_list,attentionmask_list,ids_list)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


model = transformers.BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=5)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()


def train(model, train_loader, len_train):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader)

    for data, token,attention_mask, target in train_loader:
        print(data)
        # GPUへ
        data = data.to(device)
        token = token.to(device)
        attention_mask = attention_mask.to(device)
        target = target.to(device)
        

        optimizer.zero_grad()
        pred = model(input_ids=data, token_type_ids=token, attention_mask=attention_mask)#todo modelで予測
        loss = loss_fn(pred,target)#todo lossを計算

        loss.backward()
        optimizer.step()

        total_loss += loss
        pred_labels = torch.max(pred,axis = 1).indices
        correct_num +=  (pred_labels == target).sum().item()#todo 予測の正解数をカウント

    batch_loss = total_loss/len_train #バッチごとの平均ロス
    acc = correct_num/len_train#todo 正解率
    return batch_loss, acc



len_train = len(ids_list)
for epoch in range(30):
    train_loss, train_acc = train(model, train_loader, len_train)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
    

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

RuntimeError: ignored